In [ ]:
import * as tslab from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf-8");
tslab.display.html(`<style>${css}</style>`);

In [ ]:
import { RecursiveSet, Value, Tuple } from 'recursive-set';

type Variable = string;
type Literal = Variable | Tuple<['¬', Variable]>;
type Clause = RecursiveSet<Literal>;
type Clauses = RecursiveSet<Clause>;

# The <a href="https://en.wikipedia.org/wiki/DPLL_algorithm">Davis-Putnam Algorithm</a>

This notebook implements the algorithm of Davis and Putnam.  Further details about this algorithm are provided in the lecture notes.

The function `complement(l)` computes the complement of a literal `l`.
If $p$ is a propositional variable, we have the following: 
* $\texttt{complement}(p) = \neg p$,
* $\texttt{complement}(\neg p) = p$.

As we are working with clauses that result form transforming given formulas into *conjunctive normal form* and these clauses do not contain $\top$ or $\bot$, we don't have to bother with $\top$ or $\bot$ in this function.

In [ ]:
function complement(l: Literal): Literal {
    if (typeof l === 'string') {
        return new Tuple('¬', l);
    } else {
        return l.get(1);
    }
}

The function `extractVariable(l)` extracts the variable from the literal `l`.
If $p$ is a propositional variable, we have the following: 
* $\texttt{extractVariable}(p) = p$,
* $\texttt{extractVariable}(\neg p) = p$.

In [ ]:
function extractVariable(l: Literal): Variable {
    if (typeof l === 'string') {
        return l;
    } else {
        return l.get(1);
    }
}

The function `arb(S)` returns an arbitrary element from the set `S`.

In [ ]:
function arb<T extends Value>(S: RecursiveSet<T>): T | null {
    if (S.isEmpty()) return null;
    const val = S.pickRandom(); 
    return val !== undefined ? val : null;
}

The function `selectVariable(Variables, UsedVars)`
selects an arbitrary variable from the set `Variables` that does not occur in the set `UsedVars`.  This variable is returned.

In [ ]:
function selectVariable(
    Variables: RecursiveSet<Variable>,
    UsedVars: RecursiveSet<Variable>
): Variable | null {
    for (const x of Variables) {
        if (!UsedVars.has(x)) {
            return x;
        }
    }
    return null;
}

Given a set of clauses `Clauses` and a literal `l`, the procedure `reduce(Clauses, l)` performs all unit cuts and all unit subsumptions on clauses of of the set `Clauses` that are possible using the unit clause $\{\mathtt{l}\}$.  The resulting set of clauses is returned.  Mathematically, the function `reduce` is defined as follows:
$$\texttt{reduce}(\texttt{Clauses},l)  := 
 \Bigl\{\, C \backslash \bigl\{\overline{\,l\,}\bigr\} \;|\; C \in \texttt{Clauses} \wedge \overline{\,l\,} \in C \,\Bigr\} 
       \,\cup\, \Bigl\{\, C \in \texttt{Clauses} \mid \overline{\,l\,} \not\in C \wedge l \not\in C \Bigr\} \cup \bigl\{\{l\}\bigr\}.
$$
This function should only be called if the unit clause $\{l\}$ is an element of the set `Clauses`.

In [ ]:
function reduce(Clauses: Clauses, l: Literal): Clauses {
    const lBar = complement(l);
    const result = new RecursiveSet<Clause>();

    for (const clause of Clauses) {
        if (clause.has(l)) continue;

        if (clause.has(lBar)) {
            const singletonLBar = new RecursiveSet<Literal>(lBar);
            const newClause = clause.difference(singletonLBar);
            result.add(newClause);
        } else {
            result.add(clause);
        }
    }

    result.add(new RecursiveSet<Literal>(l));
    return result;
}


`Clauses` is a set of clauses.  The call `saturate(Clauses)` computes the set of those clauses that can be derived from `Clauses` via repeated applications of unit cuts or unit subsumptions.

In [ ]:
function saturate(Clauses: Clauses): Clauses {
    let S: Clauses = Clauses;
    const Used = new RecursiveSet<Clause>();
    while (true) {
        const Units = new RecursiveSet<Clause>();
        for (const clause of S) {
            if (clause.size === 1 && !Used.has(clause)) {
                Units.add(clause);
            }
        }
        const unit = arb(Units);
        if (unit === null) break;
        Used.add(unit);
        const l = arb(unit);
        if (l === null) break;
        S = reduce(S, l);
    }
    return S;
}

The function `solve(Clauses)` takes a set of clauses  as input.  The function tries to compute a variable assignment that satisfies all clauses in `Clauses`.  If this is successful, a set of unit clauses is returned.  This set of unit clauses does not contain  any complementary literals and therefore corresponds to a variable assignment satisfying all clauses.  If the set `Clauses` is unsatisfiable, then the set `{{}}` is returned instead.

The real work is done by the function `solveRecursive`.  This function takes two additional arguments:
* `Variables` is the set of all variables occurring in `Clauses`.
* `UsedVars`  is the set of those variables that have already been used in case distinctions.

In [ ]:
function solveRecursive(
    Clauses: Clauses,
    Variables: RecursiveSet<Variable>,
    UsedVars: RecursiveSet<Variable>
): Clauses {
    const S = saturate(Clauses);
    const EmptyClause = new RecursiveSet<Literal>();

    // 1. Inkonsistenz
    if (S.has(EmptyClause)) {
        const Falsum = new RecursiveSet<Clause>();
        Falsum.add(EmptyClause);
        return Falsum;
    }

    // 2. Trivialer Fall (nur noch Unit-Clauses)
    let allUnits = true;
    for (const C of S) {
        if (C.size !== 1) {
            allUnits = false;
            break;
        }
    }

    if (allUnits) {
        return S;
    }

    // 3. Variable auswählen
    const p = selectVariable(Variables, UsedVars);
    
    // Safety check für TypeScript (sollte logisch nicht eintreten, wenn allUnits == false)
    if (p === null) {
        return S; 
    }

    const nextUsedVars = UsedVars.union(new RecursiveSet<Variable>(p));

    // Branch 1: {p}
    const unitP = new RecursiveSet<Clause>();
    const cP = new RecursiveSet<Literal>(p);
    unitP.add(cP);

    const Result1 = solveRecursive(S.union(unitP), Variables, nextUsedVars);
    if (!Result1.has(EmptyClause)) {
        return Result1;
    }

    // Branch 2: {¬p}
    const unitNotP = new RecursiveSet<Clause>();
    const cNotP = new RecursiveSet<Literal>(new Tuple('¬', p));
    unitNotP.add(cNotP);

    return solveRecursive(S.union(unitNotP), Variables, nextUsedVars);
}

In [ ]:
function solve(Clauses: RecursiveSet<Clause>): RecursiveSet<Clause> {
    const Variables = new RecursiveSet<Variable>();
    for (const clause of Clauses) {
        for (const lit of clause) {
            Variables.add(extractVariable(lit));
        }
    }
    const UsedVars = new RecursiveSet<Variable>();
    return solveRecursive(Clauses, Variables, UsedVars);
}

The function $\texttt{toString}(S)$ takes a set $S$ as input.  The set $S$ is a set of frozensets and the function converts $S$ into a string that looks like a set of sets.  This is only used for pretty printing.

In [ ]:
function literal_to_str(C: Clause): string {
    const val = arb(C);
    if (val === null) return "{}";
    if (typeof val === 'string') {
        return `${val} ↦ True`;
    } else {
        return `${val.get(1)} ↦ False`;
    }
}

function prettify(Clauses: RecursiveSet<Clause>): string {
    const res: string[] = [];
    for (const C of Clauses) res.push(C.toString());
    return `{${res.join(', ')}}`;
}

function prettifyClauses(M: Clauses): string {
    const clauseStrings: string[] = [];
    for (const clause of M) {
        const literalStrings: string[] = [];
        for (const lit of clause) {
            if (typeof lit === 'string') {
                literalStrings.push(lit);
            } else {
                literalStrings.push(`${lit.get(0)}${lit.get(1)}`);
            }
        }
        clauseStrings.push(`{${literalStrings.join(', ')}}`);
    }
    return `{${clauseStrings.join(', ')}}`;
}



function toString(S: Clauses, Simplified: Clauses): string {
    const EmptyClause = new RecursiveSet<Literal>();
    if (Simplified.has(EmptyClause)) {
        return `${prettifyClauses(S)} is unsolvable`;
    }
    const parts: string[] = [];
    for (const C of Simplified) {
        parts.push(literal_to_str(C));
    }
    return '{ ' + parts.join(', ') + ' }';
}

In [ ]:
const c1: Clause = new RecursiveSet<Literal>('r', 'p', 's');
const c2: Clause = new RecursiveSet<Literal>('r', 's');
const c3: Clause = new RecursiveSet<Literal>('p', 'q', 's');
const c4: Clause = new RecursiveSet<Literal>(new Tuple('¬', 'p'), new Tuple('¬', 'q'));
const c5: Clause = new RecursiveSet<Literal>(new Tuple('¬', 'p'), 's', new Tuple('¬', 'r'));
const c6: Clause = new RecursiveSet<Literal>('p', new Tuple('¬', 'q'), 'r');
const c7: Clause = new RecursiveSet<Literal>(new Tuple('¬', 'r'), new Tuple('¬', 's'), 'q');
const c8: Clause = new RecursiveSet<Literal>(new Tuple('¬', 'p'), new Tuple('¬', 's'));
const c9: Clause = new RecursiveSet<Literal>('p', new Tuple('¬', 'r'), new Tuple('¬', 'q'));
const c0: Clause = new RecursiveSet<Literal>(new Tuple('¬', 'p'), 'r', 'q', new Tuple('¬', 's'));

const S = new RecursiveSet<Clause>(
    c0, c1, c2, c3, c4, c5, c6, c7, c8, c9
);
console.log(toString(S, solve(S)));

In [ ]:
const c11: Clause = new RecursiveSet<Literal>('p', 'r', 'q', new Tuple('¬', 's'));
const S: RecursiveSet<Clause> = new RecursiveSet<Clause>(
    c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c11
);
console.log(toString(S, solve(S)));

In [ ]:
const c1: Clause = new RecursiveSet<Literal>('r', 'p', 's');
const c2: Clause = new RecursiveSet<Literal>('r', 's');
const c3: Clause = new RecursiveSet<Literal>('q', 'p', 's');
const c4: Clause = new RecursiveSet<Literal>(new Tuple('¬', 'p'), new Tuple('¬', 'q'));
const c5: Clause = new RecursiveSet<Literal>(new Tuple('¬', 'p'), 's', new Tuple('¬', 'r'));
const c6: Clause = new RecursiveSet<Literal>('p', new Tuple('¬', 'q'), 'r');
const c7: Clause = new RecursiveSet<Literal>(new Tuple('¬', 'r'), new Tuple('¬', 's'), 'q');
const c8: Clause = new RecursiveSet<Literal>('p', 'q', 'r', 's');
const c9: Clause = new RecursiveSet<Literal>('r', new Tuple('¬', 's'), 'q');
const c10: Clause = new RecursiveSet<Literal>('s', new Tuple('¬', 'r'), new Tuple('¬', 'q'));
const c11: Clause = new RecursiveSet<Literal>('s', new Tuple('¬', 'r'));
const c12: Clause = new RecursiveSet<Literal>('r', new Tuple('¬', 's'));

const S: RecursiveSet<Clause> = new RecursiveSet<Clause>(
    c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12
);
console.log(prettifyClauses(solve(S)));